In [46]:
import sqlite3

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from category_encoders import OrdinalEncoder

import pickle

#### 데이터 가져오기

In [47]:
# DB 연결 후 테이블 모두 join
DB_FILENAME = 'projectDB.db'
conn = sqlite3.connect(DB_FILENAME)
cur = conn.cursor()
tablejoin = cur.execute("""SELECT r.상호명, r.리뷰날짜, r.별점, 
iv.메뉴분류, iv.상세주소, iv.노키즈존여부, iv.예약가능여부, iv.룸보유여부, iv.유아서비스기타
FROM review r
JOIN (SELECT * FROM info i JOIN visitjeju v ON i.상호명 == v.상호명) AS iv
ON r.상호명 == iv.상호명;
""")

In [48]:
# join한 테이블을 데이터프레임으로 가져오기
cols = [col[0] for col in tablejoin.description]
df = pd.DataFrame(data=tablejoin.fetchall(), columns=cols)

In [49]:
# DB 연결 종료
cur.close()
conn.close()

#### 데이터 정제

In [50]:
df.head()

,상호명,리뷰날짜,별점,메뉴분류,상세주소,노키즈존여부,예약가능여부,룸보유여부,유아서비스기타
0,우유부단,2022.08.22,80,카페,제주특별자치도 제주시 한림읍 금악동길 38 1층,,,,
1,우유부단,2022.05.11,100,카페,제주특별자치도 제주시 한림읍 금악동길 38 1층,,,,
2,우유부단,2022.04.16,60,카페,제주특별자치도 제주시 한림읍 금악동길 38 1층,,,,
3,우유부단,2022.01.12,40,카페,제주특별자치도 제주시 한림읍 금악동길 38 1층,,,,
4,우유부단,2022.01.11,100,카페,제주특별자치도 제주시 한림읍 금악동길 38 1층,,,,


In [51]:
# 공백을 NaN로 변경
df = df.replace('', np.nan)

In [52]:
# df 정보 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40745 entries, 0 to 40744
Data columns (total 9 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   상호명      40745 non-null  object
 1   리뷰날짜     40186 non-null  object
 2   별점       40745 non-null  int64 
 3   메뉴분류     40745 non-null  object
 4   상세주소     40745 non-null  object
 5   노키즈존여부   13711 non-null  object
 6   예약가능여부   14335 non-null  object
 7   룸보유여부    482 non-null    object
 8   유아서비스기타  166 non-null    object
dtypes: int64(1), object(8)
memory usage: 2.8+ MB


- 결측치 처리

In [53]:
# 결측치 확인
df.isnull().sum()

상호명            0
리뷰날짜         559
별점             0
메뉴분류           0
상세주소           0
노키즈존여부     27034
예약가능여부     26410
룸보유여부      40263
유아서비스기타    40579
dtype: int64

In [54]:
# 상호명별 리뷰날짜 결측치 확인
null_rest = df[df['리뷰날짜'].isnull()]['상호명'].unique()
for rest in null_rest:
    print(f"{rest}) 날짜 O:", df[df['상호명']==rest]['리뷰날짜'].notnull().sum(), 
    ", 날짜 X:", df[df['상호명']==rest]['리뷰날짜'].isnull().sum())

치저스) 날짜 O: 3 , 날짜 X: 65
천리식당) 날짜 O: 3 , 날짜 X: 54
다람쥐 식탁) 날짜 O: 3 , 날짜 X: 45
목포고을) 날짜 O: 3 , 날짜 X: 71
글라글라하와이) 날짜 O: 3 , 날짜 X: 87
남양수산) 날짜 O: 3 , 날짜 X: 69
잇칸시타) 날짜 O: 3 , 날짜 X: 81
몬스테라 자구리) 날짜 O: 3 , 날짜 X: 57
마마무말가든) 날짜 O: 3 , 날짜 X: 30


In [55]:
# 리뷰날짜가 없는 상호/전체 비율 확인
df[df['리뷰날짜'].isnull()]['상호명'].nunique() / df['상호명'].nunique()

0.017045454545454544

In [56]:
# 비율이 적으므로 해당 상호 삭제
del_index = df[df['리뷰날짜'].isnull()].index
df = df.drop(del_index)

In [57]:
# 노키즈존여부/예약가능여부 결측치는 unknown으로 대체
df[['노키즈존여부', '예약가능여부']] = df[['노키즈존여부', '예약가능여부']].replace(np.nan, 'unknown')

In [58]:
# 룸보유여부/유아서비스기타는 결측치가 많으므로 해당 컬럼의 변별력이 없을 것으로 판단하여 삭제
df = df.drop(['룸보유여부', '유아서비스기타'], axis=1)

In [59]:
# 결측치 처리 완료
df.isnull().sum()

상호명       0
리뷰날짜      0
별점        0
메뉴분류      0
상세주소      0
노키즈존여부    0
예약가능여부    0
dtype: int64

- 중복값 확인

In [60]:
df.duplicated().sum()

7142

In [61]:
df = df.drop_duplicates()

In [62]:
df.shape

(33044, 7)

- 메뉴 대분류 추가

In [63]:
# 메뉴분류 확인
df['메뉴분류'].unique()

array(['카페', '한식', '스테이크,립', '양식', '멕시칸,브라질', '육류,고기', '샤브샤브', '회', '떡볶이',
       '해물,생선', '아이스크림', '제과,베이커리', '닭요리', '호프,요리주점', '피자', '분식', '한정식',
       '햄버거', '이탈리안', '국수', '중국요리', '굴,전복', '삼계탕', '족발,보쌈', '커피전문점',
       '해산물뷔페', '디저트카페', '일본식주점', '국밥', '돈까스,우동', '일식집', '삼겹살', '두부전문점',
       '추어', '베트남음식', '인도음식', '불고기,두루치기', '뷔페', '떡,한과', '기념품판매', '중식',
       '일본식라면', '동남아음식', '초밥,롤', '치킨', '아시아음식', '갈비', '해장국', '일식', '테마카페',
       '냉면', '도시락', '인테리어장식판매', '콘도,리조트', '해산물', '북카페', '서점', '주류제조',
       '음료,주류제조', '전통찻집', '퓨전요리', '퓨전일식', '조개', '민박', '순대', '터키음식',
       '한식뷔페', '샌드위치', '죽', '기사식당', '생과일전문점', '감자탕'], dtype=object)

In [64]:
df[(df['메뉴분류']=='기념품판매') | (df['메뉴분류']=='인테리어장식판매')
 | (df['메뉴분류']=='콘도,리조트') | (df['메뉴분류']=='민박') | (df['메뉴분류']=='서점')
 | (df['메뉴분류']=='닭요리')].groupby('상호명').head(1)

,상호명,리뷰날짜,별점,메뉴분류,상세주소,노키즈존여부,예약가능여부
972,숨비나리카페,2022.08.16,60,닭요리,제주특별자치도 서귀포시 안덕면 중산간서로1615번길 8,unknown,unknown
11407,제스토리,2022.07.11,80,기념품판매,제주특별자치도 서귀포시 막숙포로 60,unknown,unknown
18830,성미가든,2022.08.12,60,닭요리,제주특별자치도 제주시 조천읍 교래1길 2,unknown,unknown
21576,나나이로 아코제주,2022.05.17,60,인테리어장식판매,제주특별자치도 제주시 구좌읍 구좌로 75-1,unknown,unknown
22764,핀크스 비오토피아,2022.08.01,20,"콘도,리조트",제주특별자치도 서귀포시 안덕면 산록남로762번길 79,unknown,unknown
24630,달리북카페,2022.08.09,100,서점,제주특별자치도 제주시 한림읍 월계로 18 1층,unknown,unknown
26158,숨비나리,2022.08.16,60,닭요리,제주특별자치도 서귀포시 안덕면 중산간서로1615번길 8,unknown,y
28724,수망일기,2021.09.18,100,민박,제주특별자치도 서귀포시 남원읍 남조로 593-2,unknown,unknown


In [65]:
# 정보를 수집한 카카오맵을 제외한 다른 곳에서 정보 수집하여 메뉴분류 변경
# 네이버 마이플레이스 이용
df.loc[df[df['상호명']=='핀크스 비오토피아'].index, '메뉴분류'] = '양식'
df.loc[df[df['상호명']=='달리북카페'].index, '메뉴분류'] = '북카페'
df.loc[df[df['상호명']=='숨비나리'].index, '메뉴분류'] = '한식'
df.loc[df[df['상호명']=='수망일기'].index, '메뉴분류'] = '카페'

In [66]:
# 숨비나리카페/제스토리/나나이로 아코제주는 정보오류이므로 삭제
del_index = df[(df['상호명']=='숨비나리카페') | (df['상호명']=='제스토리') | (df['상호명']=='나나이로 아코제주')].index
df = df.drop(del_index)

In [67]:
# 13가지 카테고리로 메뉴분류 구분
cat1 = ['삼계탕', '한식', '한정식', '해장국', '국밥', '두부전문점', '추어', '아구', '찌개,전골', 
'매운탕,해물탕', '설렁탕', '순대', '죽', '기사식당', '감자탕', '퓨전요리', '닭요리']
cat2 = ['스테이크,립', '양식', '이탈리안', '도시락']
cat3 = ['중국요리', '중식']
cat4 = ['돈까스,우동', '일식집', '일본식라면', '초밥,롤', '일식', '퓨전일식']
cat5 = ['멕시칸,브라질', '인도음식', '베트남음식', '동남아음식', '아시아음식', '터키음식']
cat6 = ['떡볶이', '분식']
cat7 = ['치킨', '피자', '햄버거']
cat8 = ['카페', '제과,베이커리', '아이스크림', '커피전문점', '디저트카페', '샌드위치', '떡,한과', 
'생과일전문점', '테마카페', '북카페', '전통찻집']
cat9 = ['해산물', '해물,생선', '해산물뷔페', '회', '굴,전복', '게,대게', '조개']
cat10 = ['육류,고기', '샤브샤브', '불고기,두루치기', '족발,보쌈', '갈비', '삼겹살']
cat11 = ['뷔페', '한식뷔페']
cat12 = ['국수', '냉면']
cat13 = ['호프,요리주점', '일본식주점', '주류제조', '음료,주류제조']

In [68]:
for idx, item in enumerate(df['메뉴분류']):
    if item in cat1: df.loc[idx, '메뉴대분류'] = '한식'
    elif item in cat2: df.loc[idx, '메뉴대분류'] = '양식'
    elif item in cat3: df.loc[idx, '메뉴대분류'] = '중식'
    elif item in cat4: df.loc[idx, '메뉴대분류'] = '일식'
    elif item in cat5: df.loc[idx, '메뉴대분류'] = '아시아/기타'
    elif item in cat6: df.loc[idx, '메뉴대분류'] = '분식'
    elif item in cat7: df.loc[idx, '메뉴대분류'] = '치킨/피자/햄버거'
    elif item in cat8: df.loc[idx, '메뉴대분류'] = '카페/디저트'
    elif item in cat9: df.loc[idx, '메뉴대분류'] = '해산물'
    elif item in cat10: df.loc[idx, '메뉴대분류'] = '고기/구이/족발/보쌈'
    elif item in cat11: df.loc[idx, '메뉴대분류'] = '뷔페'
    elif item in cat12: df.loc[idx, '메뉴대분류'] = '국수'
    elif item in cat13: df.loc[idx, '메뉴대분류'] = '주점'
    else: print("카테고리가 없습니다.")

In [69]:
df.sample(5)

,상호명,리뷰날짜,별점,메뉴분류,상세주소,노키즈존여부,예약가능여부,메뉴대분류
40075,당케올레국수,2020.04.26,100.0,한식,제주특별자치도 서귀포시 표선면 민속해안로 584,y,n,NaN
24018,커피파인더,2021.06.23,100.0,커피전문점,제주특별자치도 제주시 서광로32길 20 1층,unknown,unknown,카페/디저트
18850,성미가든,2020.07.25,20.0,닭요리,제주특별자치도 제주시 조천읍 교래1길 2,unknown,unknown,해산물
21528,맛나식당,2019.03.14,60.0,"해물,생선",제주특별자치도 서귀포시 성산읍 동류암로 41,unknown,unknown,해산물
2380,한림칼국수,2021.05.28,100.0,국수,제주특별자치도 제주시 한림읍 한림해안로 141,unknown,unknown,해산물


In [70]:
# 메뉴대분류 생성으로 인해 발생한 결측치 다시 제거
df.isnull().sum()

상호명       6901
리뷰날짜      6901
별점        6901
메뉴분류      6901
상세주소      6901
노키즈존여부    6901
예약가능여부    6901
메뉴대분류     6901
dtype: int64

In [71]:
df = df.dropna()

In [72]:
df.isnull().sum()

상호명       0
리뷰날짜      0
별점        0
메뉴분류      0
상세주소      0
노키즈존여부    0
예약가능여부    0
메뉴대분류     0
dtype: int64

- 리뷰작성연도 추가

In [73]:
# 리뷰날짜 타입 변경
df['리뷰날짜'] = pd.to_datetime(df['리뷰날짜'])

In [74]:
# 리뷰날짜 범위 확인
df['리뷰날짜'].describe(datetime_is_numeric=True)

count                            26062
mean     2020-11-09 01:40:20.351469568
min                2015-11-24 00:00:00
25%                2020-02-22 06:00:00
50%                2021-02-28 00:00:00
75%                2021-11-24 00:00:00
max                2022-08-27 00:00:00
Name: 리뷰날짜, dtype: object

In [75]:
# 리뷰날짜 카테고리 분류: 2010년대/2020년/2021년/2022년
idx22 = df[df['리뷰날짜'].dt.year == 2022].index
idx21 = df[df['리뷰날짜'].dt.year == 2021].index
idx20 = df[df['리뷰날짜'].dt.year == 2020].index
idx10 = df[df['리뷰날짜'].dt.year < 2020].index

df.loc[idx22, '리뷰작성연도'] = 2022
df.loc[idx21, '리뷰작성연도'] = 2021
df.loc[idx20, '리뷰작성연도'] = 2020
df.loc[idx10, '리뷰작성연도'] = 2019

In [76]:
df.sample(5)

,상호명,리뷰날짜,별점,메뉴분류,상세주소,노키즈존여부,예약가능여부,메뉴대분류,리뷰작성연도
11999,윤옥,2021-08-12,100.0,일본식라면,제주특별자치도 제주시 구남동2길 19-4 혁진빌 1층,y,n,카페/디저트,2021.0
1613,키친오즈,2022-02-23,60.0,카페,제주특별자치도 제주시 한림읍 협재로 208,unknown,unknown,국수,2022.0
25337,오는정김밥,2018-06-09,20.0,분식,제주특별자치도 서귀포시 동문동로 2,unknown,unknown,고기/구이/족발/보쌈,2019.0
10938,전설의마녀,2020-07-19,20.0,"돈까스,우동",제주특별자치도 제주시 애월읍 애월해안로 715 2층,unknown,unknown,일식,2020.0
18844,성미가든,2021-04-16,100.0,닭요리,제주특별자치도 제주시 조천읍 교래1길 2,unknown,unknown,해산물,2021.0


- 위치 추가

In [77]:
# 상세주소를 통한 대략적인 위치(읍/면) 추가
locations = ['애월', '한림', '한경', '대정', '안덕', '남원', '표선', '성산', '구좌', '조천', '우도', '추자']

for loc in locations:
    tmpindex = df[df['상세주소'].str.contains(loc)].index
    df.loc[tmpindex, '위치'] = loc

In [78]:
# 추가 위치(제주시/서귀포시) 추가
noloc_df = df[df['위치'].isnull()]

locations = ['제주', '서귀포']

for loc in locations:
    tmpindex = noloc_df[noloc_df['상세주소'].str.contains(loc)].index
    df.loc[tmpindex, '위치'] = loc

In [79]:
df.sample(5)

,상호명,리뷰날짜,별점,메뉴분류,상세주소,노키즈존여부,예약가능여부,메뉴대분류,리뷰작성연도,위치
3125,낭뜰에쉼팡,2021-06-27,100.0,한식,제주특별자치도 제주시 조천읍 남조로 2343,unknown,unknown,국수,2021.0,조천
14434,오드랑베이커리,2020-11-11,80.0,"제과,베이커리",제주특별자치도 제주시 조천읍 조함해안로 552-3 1층,unknown,unknown,분식,2020.0,조천
17973,청운식당,2022-07-19,100.0,한식,제주특별자치도 서귀포시 성산읍 일출로 285 1층,unknown,y,양식,2022.0,성산
32942,카페 라라라,2021-10-02,100.0,카페,제주특별자치도 제주시 구좌읍 해맞이해안로 1430 1층,y,n,한식,2021.0,구좌
4764,생이소리,2020-05-26,100.0,한식,제주특별자치도 제주시 명림로 241,unknown,unknown,중식,2020.0,제주


In [80]:
# 위치 null값 확인
df['위치'].isnull().sum()

49

In [81]:
df[df['위치'].isnull()].head()

,상호명,리뷰날짜,별점,메뉴분류,상세주소,노키즈존여부,예약가능여부,메뉴대분류,리뷰작성연도,위치
14663,서민흑돼지,2022-06-30,60.0,갈비,대구 수성구 들안로 8-5,unknown,unknown,한식,2022.0,NaN
14664,서민흑돼지,2022-05-26,80.0,갈비,대구 수성구 들안로 8-5,unknown,unknown,한식,2022.0,NaN
14665,서민흑돼지,2022-04-24,20.0,갈비,대구 수성구 들안로 8-5,unknown,unknown,한식,2022.0,NaN
14666,서민흑돼지,2022-04-18,60.0,갈비,대구 수성구 들안로 8-5,unknown,unknown,한식,2022.0,NaN
14667,서민흑돼지,2022-04-02,100.0,갈비,대구 수성구 들안로 8-5,unknown,unknown,한식,2022.0,NaN


In [82]:
# 잘못된 데이터이므로 삭제
del_index = df[df['위치'].isnull()].index
df = df.drop(del_index)

- 모델링 시 불필요한 컬럼 삭제

In [83]:
del_colunm = ['리뷰날짜', '메뉴분류', '상세주소']
df_new = df.drop(del_colunm, axis=1)

In [84]:
df_new.sample(10)

,상호명,별점,노키즈존여부,예약가능여부,메뉴대분류,리뷰작성연도,위치
9993,톰톰카레,40.0,unknown,unknown,일식,2019.0,구좌
13954,중문신라원,20.0,unknown,unknown,국수,2022.0,서귀포
6055,신산리마을카페,80.0,unknown,unknown,해산물,2020.0,성산
28024,쉐프의스시이야기,80.0,y,y,해산물,2022.0,제주
1598,모모제이,80.0,y,y,국수,2019.0,제주
26564,무상찻집,100.0,n,unknown,카페/디저트,2021.0,제주
21727,와랑와랑카페,100.0,unknown,unknown,양식,2019.0,남원
1604,모모제이,100.0,y,y,국수,2019.0,제주
1095,보스코화덕피자,100.0,unknown,unknown,해산물,2021.0,제주
32916,카페 라라라,100.0,y,n,한식,2021.0,구좌


In [85]:
df_new.shape

(26013, 7)

In [86]:
# DB에 modelingdata 테이블로 저장
DB_FILENAME = 'projectDB.db'
conn = sqlite3.connect(DB_FILENAME)
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS modelingdata;")
df_new.to_sql('modelingdata', conn)
cur.close()
conn.close()

#### 머신러닝 모델링

- train/val/test 분리

In [87]:
# train/val/test = 0.6/0.2/0/2
train, test = train_test_split(df_new, test_size=0.4, random_state=14)
val, test = train_test_split(test, test_size=0.5, random_state=14)

train.shape, val.shape, test.shape

((15607, 7), (5203, 7), (5203, 7))

In [88]:
# target/feature
target = '상호명'
features = df_new.drop(target, axis=1).columns

X_train, y_train = train[features], train[target]
X_val, y_val = val[features], val[target]
X_test, y_test = test[features], test[target]

- 모델선정

In [89]:
# 기준모델 = 최빈값
train[target].value_counts(normalize=True)

우진해장국         0.029730
오는정김밥         0.022810
깡촌흑돼지         0.016787
오드랑베이커리       0.015442
중문수두리보말칼국수    0.014801
                ...   
잇칸시타          0.000128
마마무말가든        0.000128
다람쥐 식탁        0.000128
천리식당          0.000128
몬스테라 자구리      0.000064
Name: 상호명, Length: 416, dtype: float64

In [92]:
pipe = make_pipeline(
    OrdinalEncoder(), 
    DecisionTreeClassifier(random_state=14)
)

pipe.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['노키즈존여부', '예약가능여부', '메뉴대분류', '위치'],
                                mapping=[{'col': '노키즈존여부',
                                          'data_type': dtype('O'),
                                          'mapping': unknown    1
y          2
n          3
NaN       -2
dtype: int64},
                                         {'col': '예약가능여부',
                                          'data_type': dtype('O'),
                                          'mapping': unknown    1
y          2
n          3
NaN       -2
dtype: int64},
                                         {'col': '메뉴대분류',
                                          'data_type': dtype('O'),
                                          'mapping': 고기/구이/족발/보쌈     1
카페/디저트          2
중식              3
일식              4
한식              5
국수              6
해산물             7
아시아/기타          8
주점              9
분식             10
양식             11
치킨/피자/햄버거      12
뷔페             13
NaN            -2
dtype: int64},
                                         {'col': '위치', 'data_type': dtype('O'),
                                          'mapping': 애월      1
한림      2
제주      3
구좌      4
서귀포     5
안덕      6
대정      7
남원      8
조천      9
성산     10
한경     11
우도     12
표선     13
NaN    -2
dtype: int64}])),
                ('decisiontreeclassifier',
                 DecisionTreeClassifier(random_state=14))])

In [93]:
print('결정트리 훈련 정확도: ', pipe.score(X_train, y_train))
print('결정트리 검증 정확도: ', pipe.score(X_val, y_val))

결정트리 훈련 정확도:  0.609021592874992
결정트리 검증 정확도:  0.5406496252162214


- 모델해석

In [101]:
print('결정트리 테스트 정확도: ', pipe.score(X_test, y_test))

결정트리 테스트 정확도:  0.532961752834903


In [95]:
# X_test의 예측값과 실제값 비교
X_test_pred = pipe.predict(X_test)

jejudf_compare = pd.DataFrame({
    '예측값': X_test_pred,
    '실제값': y_test
})

jejudf_compare = jejudf_compare.join(X_test[['별점', '노키즈존여부', '예약가능여부', '메뉴대분류', '리뷰작성연도', '위치']])

In [96]:
jejudf_compare['예측비교'] = (jejudf_compare['예측값'] == jejudf_compare['실제값'])
jejudf_compare

,예측값,실제값,별점,노키즈존여부,예약가능여부,메뉴대분류,리뷰작성연도,위치,예측비교
31945,싱싱잇,싱싱잇,60.0,n,y,한식,2022.0,한림,True
23736,수카사,수카사,100.0,y,unknown,카페/디저트,2020.0,구좌,True
23872,요요무문,요요무문,80.0,y,n,카페/디저트,2020.0,구좌,True
24543,성산일출봉해송갈치전문점,성산일출봉해송갈치전문점,100.0,unknown,y,양식,2021.0,성산,True
23982,커피파인더,커피파인더,100.0,unknown,unknown,분식,2022.0,제주,True
...,...,...,...,...,...,...,...,...,...
15399,우진해장국,우진해장국,100.0,unknown,unknown,한식,2020.0,제주,True
15628,우진해장국,우진해장국,100.0,unknown,unknown,일식,2019.0,제주,True
12409,수우동,수우동,40.0,unknown,unknown,한식,2020.0,한림,True
22247,보영반점,카페 바다동굴,20.0,unknown,unknown,중식,2021.0,한림,False


In [97]:
jejudf_compare.groupby('예측비교').count()

,예측값,실제값,별점,노키즈존여부,예약가능여부,메뉴대분류,리뷰작성연도,위치
예측비교,,,,,,,,
False,2430,2430,2430,2430,2430,2430,2430,2430
True,2773,2773,2773,2773,2773,2773,2773,2773


In [98]:
# 임의의 값으로 예측해보기
# 별점: 100(고정) / 노키즈존: no / 예약여부: 무관 / 분류: 카페 / 리뷰작성연도: 무관 / 위치: 성산
test_input = pd.DataFrame({
    '별점': [100]*12,
    '노키즈존여부': ['n']*12,
    '예약가능여부': ['y', 'n', 'unknown']*4,
    '메뉴대분류': ['카페/디저트']*12,
    '리뷰작성연도': [2022, 2021, 2020, 2010]*3,
    '위치': ['성산']*12
})

In [99]:
# 예측값의 빈도가 높은 순으로 추천
test_input_pred = pipe.predict(test_input)
pd.DataFrame(test_input_pred).value_counts()

도렐               4
성산일출봉 해송갈치전문점    3
새벽숯불가든           2
그리운바다성산포         1
복자씨연탄구이          1
소라네집             1
dtype: int64

In [100]:
# 해당 모델 피클링: flask_app 폴더 내부에 생성
with open('flask_app/model.pkl','wb') as pickle_file:
    pickle.dump(pipe, pickle_file)